In [19]:
from plotly.offline import init_notebook_mode
from sqlalchemy import create_engine

from query_flow.parsers.postgres_parser import PostgresParser
from query_flow.vizualizers.query_vizualizer import QueryVizualizer



In [20]:
%load_ext autoreload
%autoreload 2

init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
con_str = 'postgresql:///etrabelsi_thesis'

## Query

In [22]:
query ="""
SELECT titles.title_id
FROM titles
INNER JOIN crew ON crew.title_id = titles.title_id
INNER JOIN people ON people.person_id = crew.person_id
WHERE genres like '%Comedy%' 
  AND name in ('Owen Wilson', 'Adam Sandler', 'Jason Segel')
"""

In [23]:
query_renderer = QueryVizualizer(parser=PostgresParser())

In [24]:
cardinality_df = query_renderer.get_cardinality_df(query, con_str)

In [25]:
query_renderer.vizualize(cardinality_df, title="Basic Flow", metrics=["actual_rows"], open_=False)


In [27]:
cardinality_df[["source","target","operation_type","actual_rows","label", "label_metadata"]]

,source,target,operation_type,actual_rows,label,label_metadata
6,0,1,Seq Scan,3446261.0,People,
5,1,3,Where,NaN,People*,"Filter condition: (people.name = ANY ('{""Owen ..."
4,2,3,Seq Scan,13651901.0,Crew,
3,3,6,Hash Join,NaN,People* ⋈ Crew,"Hash Cond ('Inner', '(crew.person_id = people...."
2,4,5,Seq Scan,2379234.0,Titles,
1,5,6,Where,NaN,Titles*,Filter condition: (titles.genres ~~ '%Comedy%'...
0,6,7,Hash Join,NaN,People* ⋈ Crew ⋈ Titles*,"Hash Cond ('Inner', '(titles.title_id = crew.t..."
